In [1]:
#
# Copyright © 2019 Sunho Kim. All rights reserved.
#

In [2]:
cd ..

/home/sunho/Documents/dev/gorani-reader/backend/dataserver


In [3]:
from pyspark.sql import SparkSession, DataFrame

spark = SparkSession\
    .builder\
    .appName('App Name')\
    .getOrCreate()
sc = spark.sparkContext

In [4]:
# parameters


In [6]:
import pyspark.sql.functions as F
from pyspark.ml.feature import CountVectorizer, Normalizer
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix

from gorani.spark import read_data_all, write_data
from gorani.utils import sparse_to_array

SIMILARITY_TYPE = 'cosine'

words = read_data_all(spark, 'words', cache = True)
books = read_data_all(spark, 'books')

cv = CountVectorizer()\
    .setInputCol('content')\
    .setOutputCol('tf')\
    .setVocabSize(words.count())
cv = cv.fit(books)

nor = Normalizer()\
    .setInputCol('tf')\
    .setOutputCol('norm')

# normalized book-wordcount matrix
tf_mat = nor.transform(cv.transform(books))

# convert to SparseMatrix to BlockMatrix
mat = IndexedRowMatrix(
tf_mat.select("id", "norm")\
    .rdd.map(lambda row: IndexedRow(row['id'], row['norm'].toArray())))\
    .toBlockMatrix()

# cosine similarity
sim_mat_df = mat.multiply(mat.transpose())\
    .toIndexedRowMatrix()\
    .rows.toDF()

# change schema
df = sim_mat_df.select(F.col('index').alias('id'), F.posexplode(sparse_to_array('vector')))\
    .select('id', F.col('pos').alias('other_id'), F.col('col').alias('value'))\
    .withColumn('type', F.lit(SIMILARITY_TYPE))\
    .where('id != 0 AND other_id != 0')

write_data('book_similarity', df)
print('success')

Py4JJavaError: An error occurred while calling o432.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 47.0 failed 4 times, most recent failure: Lost task 0.3 in stage 47.0 (TID 127, 183.99.225.238, executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/etc/spark/python/lib/pyspark.zip/pyspark/worker.py", line 361, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/etc/spark/python/lib/pyspark.zip/pyspark/worker.py", line 236, in read_udfs
    arg_offsets, udf = read_single_udf(pickleSer, infile, eval_type, runner_conf)
  File "/etc/spark/python/lib/pyspark.zip/pyspark/worker.py", line 163, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/etc/spark/python/lib/pyspark.zip/pyspark/worker.py", line 64, in read_command
    command = serializer._read_with_length(file)
  File "/etc/spark/python/lib/pyspark.zip/pyspark/seri